<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Installing required libraries
# !pip install torchinfo
# import torchinfo
!pip install pytorch-minimize
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.2 MB/s eta 0:00:00


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [3]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.conv1(x)

    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)



    return x



model = LeNet5().to(device)
torch.nn.init.xavier_uniform_(model.conv1.weight)
torch.nn.init.xavier_uniform_(model.conv2.weight)
torch.nn.init.xavier_uniform_(model.fc1.weight)
torch.nn.init.xavier_uniform_(model.fc2.weight)
torch.nn.init.xavier_uniform_(model.fc3.weight)



Parameter containing:
tensor([[-1.6266e-01,  1.4402e-01, -2.4872e-01, -2.3892e-01,  5.8217e-02,
         -1.2274e-01, -1.5620e-01,  1.4433e-01,  7.7499e-02, -1.5581e-01,
          1.2261e-01,  1.5671e-01,  1.7872e-01,  2.4279e-01, -8.7386e-02,
          8.9809e-02, -1.1898e-01, -6.8837e-02, -2.5565e-03,  9.2045e-02,
         -1.8321e-01, -4.7986e-02,  1.5248e-02,  1.0698e-01,  1.9954e-01,
         -1.1392e-01,  1.6589e-02, -2.0039e-01,  2.0894e-01, -5.5351e-02,
         -1.0115e-01,  8.2095e-02,  3.0908e-02, -2.2902e-01,  9.2432e-02,
          9.8669e-02, -2.4258e-01,  8.3978e-03, -2.2438e-01, -1.2905e-02,
          2.4736e-02,  2.3419e-01,  1.9899e-01, -2.1639e-01,  8.5183e-02,
         -3.0988e-02,  1.9282e-01, -2.4545e-01,  2.4043e-01, -1.4035e-01,
          1.8520e-01, -2.0123e-01,  1.6688e-02,  4.1007e-02,  8.7395e-02,
         -2.4793e-01,  4.3835e-02,  4.7252e-02,  7.7107e-02, -1.7631e-01,
         -1.6404e-01, -2.4897e-01, -4.4764e-02, -8.4548e-02,  1.5567e-01,
          9.3926

In [4]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=False, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data, shuffle=True)

100%|██████████| 170498071/170498071 [00:02<00:00, 79362730.67it/s]


Extracting ./train/cifar-10-python.tar.gz to ./train/


In [5]:
import torchmin

criterion = nn.CrossEntropyLoss()
optimizer = torchmin.Minimizer(model.parameters(),method='l-bfgs')


In [6]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)


    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(model.parameters(),1.0e-2)

      return loss
    optimizer.step(closure)
    loss = closure()
    epoch_loss += loss.item()

  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))



Epoch: 0 Loss: 0.05615426052015457
Epoch: 1 Loss: 0.0
Epoch: 2 Loss: 0.0
Epoch: 3 Loss: 0.0
Epoch: 4 Loss: 0.0


In [9]:
y_pred = model(train_data[2][0].unsqueeze(0).to(device))
print(f" Model predicted label : {np.argmax(y_pred.detach().cpu().numpy())}")
print(f" Actual label : {train_data[2][1]}")

 Model predicted label : 1
 Actual label : 8
